# Project 3 pt. 2 - Decision Stump Construction via ID3
### By Russell Marvin

Here are our functions for entropy and InfoGain:

In [86]:
import numpy as np
#opening file, we can change this to input("what file?") if we want it to use different files
file_path = open("treatment.txt")
#use numpy genfromtxt to turn .txt into arrays, dtype specifies string values (vs int, float, etc)
data = np.genfromtxt(file_path, dtype = str)
#defining all data in an array without names of cols
all_data = data[1:,:]
#print(all_data)


def entropy(data):
    #count how frequent each outcome is using a dict
    count_freq = {}
    for instance in data:
        value = instance[0]
        #add 1 to the count for this value of the target variable
        if value in count_freq:
            count_freq[value] +=1
        #add a count = 1 and a new key corresponding w/ this unseen value of target var
        else:
            count_freq[value] = 1
    total_examples = len(data)
    #calculate probabilities
    probabilities = [i / total_examples for i in count_freq.values()]
    #loop through probabilities and apply prob*log2(prob), then calculate neg sum of them
    entropy = -sum(prob * np.log2(prob) for prob in probabilities)
    return entropy


def information_gain(data, attr_index):
  # Define the attribute and label columns for use
    attr_col = data[:, attr_index]
    target_col = data[:, 0]
    #Count the frequency of each attribute value
    unique_values, value_counts = np.unique(attr_col, return_counts=True)
    #compute the entropy of the whole dataset (target or 'Oracle' column)
    set_entropy = entropy(target_col)
    #count examples
    total_examples = len(data)
    #use a list to store entropy values to be multiplied by weight
    temp_entropies = []
    #save these intermediate entropy values in a dictionary to print later
    dict_ent = {}
    #iterate through the lists together (same length of 2) and calculate the entropy for each subset
    for val, count in zip(unique_values, value_counts):
        #examine only the rows for each unique value at a time (subset), use its corresponding count to calculate its weight
        subset = data[attr_col == val]
        #how much of the data does this subset constitute?
        weight = count / total_examples
        temp_entropy = entropy(subset[:, 0])
        #store these entropy values in our temporary dictionary
        dict_ent[val] = temp_entropy
        #add to list each weighted entropy 
        temp_entropies.append(weight * temp_entropy)
    total_entropies = np.sum(temp_entropies)
    # calculate information gain using summed temporary entropies
    info_gain = set_entropy - total_entropies

    return info_gain, attr_index

Now, let's create an implementation of the ID3 for a decision stump - implement the first level of the ID3 algorithm for decision tree construction. In other
words, perform all steps that stop short of the recursive call in the original algorithm.

Modified ID3 (S)
 - choose the attribute a that maximizes the Information Gain of S
 -  let attribute a be the decision for the root node
 - add a branch from the root node for each possible value v of attribute a 
 - for each branch:
     - “sort” examples down the branches based on their value v of attribute a 
 - classify each terminal node with its majority label

This `modified_id3()` function takes a labeled classification dataset `my_data` in the form of a string dictating the file path and a `Yes` and `No` as input, with Yes and No corresponding to what your dataset uses to indicate classification - 1/0, Yes/No, Pos/Neg, etc. 

In [87]:
def modified_id3(my_data, Yes, No):
    file_path = open(my_data)
    #use numpy genfromtxt to turn .txt into arrays, dtype specifies string values (vs int, float, etc)
    data = np.genfromtxt(file_path, dtype = str)
    #defining labels of our dataset
    labels = data[0,:]
    #defining all data in an array without names of cols
    data = data[1:,:]
    #let's save our attribute_cols and info gain in a dict
    ig_dict = {}
    #for the value of every col in our data...
    for i in range(data.shape[1]):
        #calculate the infogain of each col and save their index
        ig, col = information_gain(data,i)
        #ignore the first column (target vals, not an attr)
        if col != 0:
            ig_dict[col] = ig
            #print the intermediate value (infogain) for each attribute
            print(ig, f'is the infogain of {labels[col]}')
    best_col = (max(ig_dict, key = ig_dict.get))
    print(f'The decision node is {labels[best_col]}, aka the best attribute to choose.')
    #Now I need to grab the index (row) of all values of each v for this a
    #and assign them, classify based on majority
    values = np.unique(data[:,best_col])
    #this dict stores Yes/No counts for each subset with attr val v
    temp_dict = {}
    #this dict stores list of row index values belonging to each attr value v
    temp_dict2 = {}
    for value in values:
        #initialize each value to 0
        temp_dict[value] = {Yes:0,No:0}
        #initialize list for each value
        temp_dict2[value] = []
        #select only items with 'value' as their value
        index = np.where(data == value)[0]
        #for each row in that attribute, 
        for i in index:
            #if the target label is Yes...
            if data[i,0] == Yes:
                #I have to add 1 to i here so it fits the naming schema in the HW/tutorial
                #where 'data 1' is the first example and so on... It's actually the second 
                #row in the dataset and the 0th in index. This is trivial, but just for the 
                #sake of comprehension and validation of results, I'll add 1
                temp_dict[value][Yes] +=1
                #add that index value i to the list for this attr value v
                temp_dict2[value].append(i + 1)
            elif data[i,0] == No:
                temp_dict[value][No] +=1
                temp_dict2[value].append(i + 1)
    #print(temp_dict2)
    for k,v in temp_dict.items():
        numerator = v[Yes]
        denominator = numerator + v[No]
        if denominator == 0:
            proportion_yes == 0
        else:
            proportion_yes = numerator/denominator
        if proportion_yes >.5:
            stump = print(f'One branch is: {k}, which is labeled: {Yes}, including data instances: {temp_dict2[k]}')
        elif proportion_yes <.5:
            stump = print(f'One branch is: {k}, which is labeled: {No}, including data instances: {temp_dict2[k]}')
        elif proportion_yes == .5:
            stump = print(k,"Undecided, due to 50/50 split")  
    return stump


This function creates a sentence describing each branch in the stump (including the decision node) and the label for its constituent members (with them listed as well). This is an easily readable way to display the results as an alternative to creating a tree/stump. 

Let's try it on our treatment data from HW7:

In [88]:
modified_id3("treatment.txt", "Pos", "Neg")

0.01997309402197489 is the infogain of Pulse
0.9709505944546686 is the infogain of BP
0.4199730940219749 is the infogain of Age
The decision node is BP, aka the best attribute to choose.
One branch is: Abnormal, which is labeled: Neg, including data instances: [4, 5]
One branch is: Normal, which is labeled: Pos, including data instances: [1, 2, 3]


Now on the fishing data from the tutorial:

In [89]:
modified_id3("fishing.txt", "Yes", "No")

0.12808527889139443 is the infogain of Wind
0.06105378373381021 is the infogain of Air
0.12808527889139443 is the infogain of Water
0.2638091738835462 is the infogain of Sky
The decision node is Sky, aka the best attribute to choose.
One branch is: Cloudy, which is labeled: Yes, including data instances: [3]
One branch is: Rainy, which is labeled: No, including data instances: [4, 5, 6, 10, 14]
One branch is: Sunny, which is labeled: Yes, including data instances: [1, 2, 7, 8, 9, 11, 12, 13]


And finally on this new donors dataset:

In [90]:
modified_id3("donors.txt", "Yes", "No")

0.17549700417502345 is the infogain of Status
0.0031147587638322705 is the infogain of Party
The decision node is Status, aka the best attribute to choose.
One branch is: Family, which is labeled: Yes, including data instances: [3, 6, 9, 10, 14, 16, 20]
One branch is: Married, which is labeled: No, including data instances: [2, 4, 5, 7, 8, 12, 13, 17, 18, 19]
One branch is: Single, which is labeled: No, including data instances: [1, 11, 15]


### Problems/Data structures: 

This portion of the project went well. I felt I was able to generalize this well to each dataset, using the unique target variable labels (referred to as `Yes` and `No` in my `modified_id3()` function). I also changed made the file path dynamic such that the user can input a string indicating the file path into this function. This, combined with the aforementioned labels as inputs makes the function generally applicable to data with the same format (row/col positioning of labels, target, etc.) but with variable numbers of attributes and attribute values num. 

I used dictionaries for my `modified_id3` because I found them the most intuitive way to store my data (to access later on). I used `numpy.where()` to select certain values and found [this](https://www.digitalocean.com/community/tutorials/python-numpy-where) to be more useful than the documentation page. 

The format I decided on with print statements and sentences describin the stump is somewhat difficult to interpret, I'd prefer reading an actual tree/stump.